In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, InputLayer
from skimage.color import rgb2lab, lab2rgb
from IPython.display import display
from ipywidgets import interact, IntSlider, Layout

In [ ]:
#On charge les images en PIL dans sorties
dataset_path = '../../datasets/faces/'
images = os.listdir(dataset_path)[:30]
sorties = [load_img(dataset_path + image) for image in images]

#Possible resize
sorties = [image.resize((128, 128)) for image in sorties]

In [ ]:
#On transforme en LAB
sorties = [rgb2lab(img_to_array(image) / 255) for image in sorties]

#On isole les teintes de gris pour entrees
#et les teintes de couleurs pour sorties
entrees = [image[:, :, 0] for image in sorties]
sorties = [image[:, :, 1:] for image in sorties]

#On reshape et on renormalise
entrees = [image.reshape(image.shape + (1,))/128 for image in entrees]
sorties = [image/128 for image in sorties]

#On convertit en np array
entrees = np.array(entrees)
sorties = np.array(sorties)

In [ ]:
models = [Sequential(), Sequential()]

for model in models:
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(1, (3, 3), activation='tanh', padding='same'))

models[0].summary()

In [ ]:
for model in models:
    model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
def generator(x, y, epochs, batch_size):
    for p in range(epochs):
        for i in range(0, len(x), batch_size):
            j = i + batch_size
            yield (x[i:j], y[i:j])

In [ ]:
batch_size = 1
epochs = 1000

for i, model in enumerate(models):
    model.fit_generator(generator(entrees, sorties[:, :, :, i:i+1], epochs, batch_size),
                        epochs=epochs, verbose = 0,
                        steps_per_epoch=len(entrees)/batch_size)

In [ ]:
def prediction_a_img(models, entree):
    A = models[0].predict(entree, batch_size=1)
    B = models[1].predict(entree, batch_size=1)
    AB = np.append(A, B, axis=3)
    LAB = np.append(entree, AB, axis=3) * 128
    LAB = LAB.reshape(LAB.shape[1:])
    return array_to_img(lab2rgb(LAB))

In [ ]:
def print_sel(SEL):
    display(prediction_a_img(models, entrees[SEL:SEL+1]))

    grayscale = np.append(entrees[SEL], np.zeros(entrees[SEL].shape[:2] + (2,)), axis=2) * 128
    display(array_to_img(lab2rgb(grayscale)))

    colored = np.append(entrees[SEL], sorties[SEL], axis=2) * 128
    display(array_to_img(lab2rgb(colored)))

In [ ]:
interact(print_sel, SEL=IntSlider(min=0,max=len(entrees)-1,step=1,value=0, layout=Layout(width='90%')));